In [1]:
# However we do not expect the reader to add that folder to the env variable,
# therefore we manually load it temporarily in each notebook.
import os, sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import pandas as pd
import numpy as np
from modules.config import (
    PATH_SCENARIOS_REDUCED,
    PATH_DISTANCES,
    PATH_SCENARIO_PROBABILITY,
    N_REDUCED_SCNEARIOS,
    ALL_VEHICLE_TYPES,
    PATH_RESULTS_SUMMARY,
    PATH_RESULTS_SINGLE_MODAL_BENCHMARK,
    PATH_RESULTS_VALUE_STOCHASTIC,
    PATH_FLEET_SIZE,
    PATH_SCENARIO_TREE_NODES,
)
from modules.stochastic_program.factory import StochasticProgramFactory, StochasticProgram



# Stochastic Program
In this notebook we will use the previously prepared data to create a stochastic program and solve it. In order to evaluate its performance we will perform multiple benchmarks.
## Read Input Data

In [3]:
scenarios = pd.read_pickle(PATH_SCENARIOS_REDUCED)

In [4]:
node_df = pd.read_pickle(PATH_SCENARIO_TREE_NODES)

In [5]:
distances = pd.read_pickle(PATH_DISTANCES)

In [6]:
probabilities = pd.read_pickle(PATH_SCENARIO_PROBABILITY)

In [7]:
real_fleet_size = pd.read_pickle(PATH_FLEET_SIZE).to_dict()["id"]
# TODO
real_fleet_size = {
    vehicle_type: int(fleet_size/3) for vehicle_type, fleet_size in real_fleet_size.items()
}
real_fleet_size

{'bicycle': 996, 'car': 658, 'kick_scooter': 2600}

# Create & Solve Stochastic Program

To create and solve the stochastic program we use the `StochasticProgramFactory` and `StochasticProgram` classes from our own module.  
We first create the stochastic program factory with all of the prepared data.

In [8]:
vehicle_types = ALL_VEHICLE_TYPES
factory = StochasticProgramFactory(
    scenarios,
    distances.apply(lambda x: round(x, 2)), # rounding improves performance
    probabilities,
    node_df,
    vehicle_types,
)
factory.set_initial_allocation(real_fleet_size)


_convert_probabilities finished in 0.00 seconds
_convert_distances finished in 0.02 seconds
_convert_demand finished in 0.38 seconds
_convert_nodes finished in 0.00 seconds
_convert_parameters finished in 0.40 seconds
_set_max_demand finished in 0.12 seconds
set_initial_allocation finished in 0.01 seconds


Now we create the stochastic program using the factory. The factory will pass all the necessary data in the correct format to the stochastic program class.

In [9]:
stochastic_program: StochasticProgram = factory.create_stochastic_program()

create_stochastic_program finished in 0.01 seconds


We now use the stochastic program class to create the constraints and the objective function using the pulp library.  
We also can configure the model to be risk averse by setting a beta larger than zero. Beta is the weight of the calculate-value-at-risk. Look at the documentation in `stochastic_program.py` for further details.

In [10]:
stochastic_program.create_model(beta=0.5, alpha=0.75)

_create_variables finished in 2.43 seconds
_create_objectives finished in 5.12 seconds
_create_demand_constraints finished in 4.81 seconds
_create_relocation_binary_constraints finished in 0.03 seconds
_create_big_u_sum_constraints finished in 0.64 seconds
_create_unfulfilled_demand_binary_constraints finished in 0.04 seconds
_create_no_refused_demand_constraints finished in 0.03 seconds
_create_relocations_constraints finished in 1.89 seconds
_create_vehicle_trips_starting_constraints finished in 0.74 seconds
_create_vehicle_trips_ending_constraints finished in 0.75 seconds
_create_initial_allocation_constraints finished in 0.01 seconds
_create_non_anticipativity_constraints finished in 2.95 seconds
_create_value_at_risk_constraints finished in 2.62 seconds
_create_constraints finished in 14.52 seconds
create_model finished in 28.33 seconds


We now can solve the formulated program. Make sure that the solver selected in `config.py` is configured properly.

In [11]:
stochastic_program.solve()

Academic license - for non-commercial use only - expires 2021-07-23
Using license file /home/moritz/licenses/gurobi.lic
No parameters matching '_test' found
Status: Optimal
Optimal Value of Objective Function:  -7538.581250000008
solve finished in 42.57 seconds


Here we can take a look at the dimensions of the problem.

In [12]:
scenarios.reset_index().nunique()

scenarios          8
start_hex_ids     82
end_hex_ids       82
time               2
vehicle_types      3
demand           130
dtype: int64

In [13]:
stochastic_program.get_summary()

get_results_by_tuple_df finished in 2.38 seconds
get_results_by_region_df finished in 0.03 seconds
get_summary finished in 3.28 seconds


{'status': 'Optimal',
 'objective': -7538.581250000008,
 'expected_profit': -7421.772500000014,
 'eta': -7655.390000000003,
 'beta': 0.5,
 'alpha': 0.75,
 'n_trips_avg': 5041.75,
 'n_unfilled_demand_avg': 4532.0,
 'demand_avg': 9573.75,
 'n_parking_avg': 3466.25,
 'n_relocations_avg': 4727.0}

# Benchmarks
We will now run different benchmarks on our model to evaluate its performance.
## Different Capacities/ Disabled Relocations/ Value Of Perfect Information
In our first benchmark we will solve the model with 3 different vehicle fleet sizes to see how the fleet size affects the performance of the model.  
For each fleet size we will also enable/disable relocations, so that we can see how relocations can improve profit and demand fulfillment. We will also enable/disable the non-anticipativty constraints to calculate the value of perfect information.

In [14]:
capacities = [
    {
        "kick_scooter": 10000,
        "bicycle": 4000,
        "car": 3000,
    },
    {
        "kick_scooter": 8000,
        "bicycle": 3000,
        "car": 2000,
    },
    {
        "kick_scooter": 6000,
        "bicycle": 2000,
        "car": 1000,
    },
]


In [15]:
results = []

factory = StochasticProgramFactory(scenarios, distances, probabilities, node_df)
factory.include_methods = [None]
for capacity in capacities:
    factory.set_initial_allocation(capacity)

    stochastic_program = factory.create_stochastic_program()
    stochastic_program.include_methods = ['solve']

    for relocations_disabled in [False, True]:
        for non_anticipativity_disabled in [False, True]:
            stochastic_program.relocations_disabled = relocations_disabled
            stochastic_program.non_anticipativity_disabled = non_anticipativity_disabled
            stochastic_program.create_model()
            stochastic_program.solve()

            results.append({
                **stochastic_program.get_summary(),
                **capacity,
                'relocations_disabled': relocations_disabled,
                'non_anticipativity_disabled': non_anticipativity_disabled,
            })
            print('\n')

_convert_probabilities finished in 0.00 seconds
_convert_distances finished in 0.01 seconds
_convert_demand finished in 1.31 seconds
_convert_nodes finished in 0.01 seconds
_convert_parameters finished in 1.33 seconds
_set_max_demand finished in 0.06 seconds
Status: Optimal
Optimal Value of Objective Function:  -146486.54212865568
solve finished in 154.34 seconds


Status: Optimal
Optimal Value of Objective Function:  -141489.89986630008
solve finished in 30.51 seconds


Status: Optimal
Optimal Value of Objective Function:  -193666.55020627836
solve finished in 15.42 seconds


Status: Optimal
Optimal Value of Objective Function:  -193487.37003242382
solve finished in 12.96 seconds


Status: Optimal
Optimal Value of Objective Function:  -91784.631543927
solve finished in 188.95 seconds


Status: Optimal
Optimal Value of Objective Function:  -87883.37045716598
solve finished in 22.33 seconds


Status: Optimal
Optimal Value of Objective Function:  -131444.1195817291
solve finished in 15.2

In [16]:
results_df = pd.DataFrame.from_dict(results)

In [17]:
results_df

,status,objective,expected_profit,n_trips_avg,n_unfilled_demand_avg,demand_avg,n_parking_avg,n_relocations_avg,kick_scooter,bicycle,car,relocations_disabled,non_anticipativity_disabled
0,Optimal,-146486.542129,-146486.542129,7708.875,1864.875,9573.75,26291.125,6339.000,10000,4000,3000,False,False
1,Optimal,-141489.899866,-141489.899866,8047.500,1526.250,9573.75,25952.500,6377.125,10000,4000,3000,False,True
2,Optimal,-193666.550206,-193666.550206,5430.000,4143.750,9573.75,28570.000,0.000,10000,4000,3000,True,False
3,Optimal,-193487.370032,-193487.370032,5430.250,4143.500,9573.75,28569.750,0.000,10000,4000,3000,True,True
4,Optimal,-91784.631544,-91784.631544,7385.875,2187.875,9573.75,18614.125,5198.000,8000,3000,2000,False,False
5,Optimal,-87883.370457,-87883.370457,7707.250,1866.500,9573.75,18292.750,5133.500,8000,3000,2000,False,True
6,Optimal,-131444.119582,-131444.119582,4632.750,4941.000,9573.75,21367.250,0.000,8000,3000,2000,True,False
7,Optimal,-131359.164064,-131359.164064,4632.750,4941.000,9573.75,21367.250,0.000,8000,3000,2000,True,True
8,Optimal,-41154.367680,-41154.367680,6920.875,2652.875,9573.75,11079.125,4593.000,6000,2000,1000,False,False
9,Optimal,-38785.852647,-38785.852647,7209.500,2364.250,9573.75,10790.500,4550.625,6000,2000,1000,False,True


In [18]:
os.makedirs(os.path.dirname(PATH_RESULTS_SUMMARY), exist_ok=True)
results_df.to_pickle(PATH_RESULTS_SUMMARY)


## Single Modal Benchmark
The novelty of our model is the consideration of multiple vehicle types. With this benchmark we will examine whether this consideration actually improves the profit. To do that we will create three single modal models and run them subsequently. The first single modal model will only consider kick scooters. We will then extract the unfulfilled demand from the solution of the first model and use that as an additional input for the second single modal model, which considers bicycles. Therefore the second model will have the bicycle demand plus the unfulfilled demand from the kick scooters as input. We will then repeat the process for cars.  
We can then compare the sum of the profit of all three single modal models with the profit of the multi modal model. 

In [19]:
scenarios_copy = scenarios.copy().reset_index()

demand_per_type = {
    "car": scenarios_copy[scenarios_copy['vehicle_types'] == 'car']\
        .set_index(['scenarios','start_hex_ids','end_hex_ids','time' ,'vehicle_types']),
    "kick_scooter": scenarios_copy[scenarios_copy['vehicle_types'] == 'kick_scooter']\
        .set_index(['scenarios','start_hex_ids','end_hex_ids','time' ,'vehicle_types']),
    "bicycle": scenarios_copy[scenarios_copy['vehicle_types'] == 'bicycle']\
        .set_index(['scenarios','start_hex_ids','end_hex_ids','time' ,'vehicle_types']),
}

In [20]:
results = []
# last demand in first iteration is 0
last_demand = pd.DataFrame(index=demand_per_type['car'].index)
last_demand['demand'] = 0
for vehicle_types in [["kick_scooter"], ["bicycle"], ["car"]]:
    current_vehicle_type = vehicle_types[0]
    print("previous demand", demand_per_type[current_vehicle_type]['demand'].values.sum(), "\n")
    print("demand for current vehicle", last_demand['demand'].values.sum(), "\n")
    current_demand = demand_per_type[current_vehicle_type] \
                    .reset_index('vehicle_types') \
                    .drop("vehicle_types", axis=1) \
                    + last_demand.reset_index('vehicle_types').drop("vehicle_types", axis=1)
    current_demand['vehicle_types'] = current_vehicle_type
    current_demand = current_demand.set_index('vehicle_types', append=True)
    print("total current demand", current_demand['demand'].values.sum(), "\n")
    current_fleet_capacity = {
        current_vehicle_type: real_fleet_size[current_vehicle_type]
    }

    current_demand.index = current_demand.index.set_levels(
        current_demand.index.get_level_values("vehicle_types").map(
            lambda x: current_vehicle_type
        ),
        verify_integrity=False,
        level="vehicle_types",
    )

    factory = StochasticProgramFactory(
        current_demand,
        distances,
        probabilities,
        node_df,
        vehicle_types,
        include_methods=[None],
    )
    factory.set_initial_allocation(real_fleet_size)
    stochastic_program = factory.create_stochastic_program()
    stochastic_program.include_methods = ["solve"]
    stochastic_program.create_model()
    stochastic_program.solve()
    # we transform the unfulfilled demand of the current lp into the demand for the next lp
    last_demand = stochastic_program.get_unfulfilled_demand().rename(
        columns={"accumulated_unfulfilled_demand": "demand"}
    )
    
    results.append(
        {**stochastic_program.get_summary(), "vehicle_types": str(vehicle_types)}
    )


previous demand 48603 

demand for current vehicle 0 

total current demand 48603 

Status: Optimal
Optimal Value of Objective Function:  -3260.0242262060024
solve finished in 5.75 seconds


NameError: name 'test' is not defined

In [14]:
factory = StochasticProgramFactory(
    scenarios,
    distances,
    probabilities,
    node_df,
    ALL_VEHICLE_TYPES,
    include_methods=[None],
)
factory.set_initial_allocation(real_fleet_size)
stochastic_program = factory.create_stochastic_program()
stochastic_program.include_methods = ["solve"]
stochastic_program.create_model()
stochastic_program.solve()

results.append(
    {**stochastic_program.get_summary(), "vehicle_types": ALL_VEHICLE_TYPES}
)


Status: Optimal
Optimal Value of Objective Function:  -7279.716174043086
solve finished in 29.51 seconds


In [15]:
results = pd.DataFrame.from_dict(results)

In [17]:
compare = results.iloc[[3]]
compare.append(results.iloc[range(3)].sum(), ignore_index=True)

,status,objective,expected_profit,n_trips_avg,n_unfilled_demand_avg,demand_avg,n_parking_avg,n_relocations_avg,vehicle_types
0,Optimal,-7279.716174,-7279.716174,5004.25,4569.500,9573.750,3503.75,3576.0,"[kick_scooter, bicycle, car]"
1,OptimalOptimalOptimal,-14081.660421,-14081.660421,5062.75,12461.125,17523.875,3445.25,4065.0,['kick_scooter']['bicycle']['car']


In [ ]:
os.makedirs(os.path.dirname(PATH_RESULTS_SINGLE_MODAL_BENCHMARK), exist_ok=True)
compare.to_pickle(PATH_RESULTS_SINGLE_MODAL_BENCHMARK)

## Value Of The Stochastic Solution

In [ ]:
demand = scenarios.copy()
demand = (
    demand.unstack("scenarios")["demand"]
    .sum(axis=1)
    .to_frame()
    .rename(columns={0: "demand"})
)
demand["scenarios"] = 0
demand = demand.set_index("scenarios", append=True).reorder_levels(
    ["scenarios", "start_hex_ids", "end_hex_ids", "time", "vehicle_types"]
)
demand['demand'] = demand['demand'] / N_REDUCED_SCNEARIOS
demand['floored'] = demand['demand'].apply(np.floor)
demand['ceiled'] = demand['demand'].apply(np.ceil)


In [ ]:
results = []
for rounding_mode in ["floored", "ceiled"]:
    factory = StochasticProgramFactory(
        demand[[rounding_mode]].rename(columns={rounding_mode: "demand"}),
        distances,
        probabilities,
        node_df,
        ALL_VEHICLE_TYPES,
        include_methods=[None],
    )
    factory.set_initial_allocation(real_fleet_size)
    stochastic_program = factory.create_stochastic_program()
    stochastic_program.include_methods = ["solve"]
    
    # assign all weight to first scenario
    stochastic_program.weighting = {0: 1}

    # discard non-anticipativity constraints
    stochastic_program.non_anticipativity_disabled = True

    stochastic_program.create_model()
    stochastic_program.solve()

    results.append(
        {**stochastic_program.get_summary(), "vehicle_types": ALL_VEHICLE_TYPES}
    )


Status: Optimal
Optimal Value of Objective Function:  659.3184262029068
Runtime without preprocessing: 1.92 seconds
solve finished in 3.06 seconds
Status: Optimal
Optimal Value of Objective Function:  1101.9133428664495
Runtime without preprocessing: 1.95 seconds
solve finished in 3.45 seconds


In [ ]:
results = pd.DataFrame.from_dict(results)
results

,status,objective,solver_runtime,n_trips_avg,n_unfilled_demand_avg,demand_avg,n_parking_avg,n_relocations_avg,vehicle_types
0,Optimal,659.318426,1.918428,429.0,8545.0,8974.0,0.0,271.0,"[kick_scooter, bicycle, car]"
1,Optimal,1101.913343,1.945398,441.0,10422.0,10863.0,0.0,259.0,"[kick_scooter, bicycle, car]"


In [ ]:
results_df.to_pickle(PATH_RESULTS_VALUE_STOCHASTIC)